In [1]:
!pip install scikit-surprise

# Sistema de recomondacion

### Librerias

In [76]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise.accuracy import rmse


import pickle
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import pandas as pd

### Rutas de los archivos

In [2]:
RATINGS = '/content/RATINGS.csv'
ANIMES = '/content/ANIME.csv'

In [3]:
df_animes = pd.read_csv(ANIMES)
df_rating = pd.read_csv(RATINGS).sample(frac=0.10, random_state=1)

In [4]:
df_rating

,Unnamed: 0,user_id,anime_id,rating
5263111,5263118,65515,5114,10
3531005,3531012,43118,25183,7
4901408,4901415,60604,6377,9
428387,428387,6112,11887,7
5241989,5241996,65212,16706,8
...,...,...,...,...
2626458,2626458,31202,12883,6
4945058,4945065,61061,9617,7
5110427,5110434,63199,1726,9
14579,14579,614,889,7


In [10]:
df_rating.columns

Index(['Unnamed: 0', 'user_id', 'anime_id', 'rating'], dtype='object')

# Metricas para el modelo

# Genracion del modelo

In [11]:
data = Dataset.load_from_df(df_rating[['user_id', 'anime_id', 'rating']],
                            Reader(rating_scale=(1, 10)))

In [12]:
trainset, testset = train_test_split(data, test_size=0.2)

In [13]:
model = KNNBasic(sim_options={'user_based': True})
model.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [14]:
predictions = model.test(testset)
accuracy = rmse(predictions)

RMSE: 1.4322


In [15]:
accuracy

1.4322332607416277

# Prueba del modelo

In [16]:
df_rating.head(1)

,Unnamed: 0,user_id,anime_id,rating
5263111,5263118,65515,5114,10


In [17]:
df_animes['ANIME_ID'].dtype

dtype('int64')

In [18]:
def predict(user_id:int, model:KNNBasic, df_recommendation:pd.DataFrame, column_id:str ,n_sugestion:int = 5) -> pd.DataFrame:
  recommendations = model.get_neighbors(trainset.to_inner_uid(user_id), k=n_sugestion)
  return df_recommendation[df_recommendation[column_id].isin(recommendations)]


In [51]:
df_rating

,Unnamed: 0,user_id,anime_id,rating
5263111,5263118,65515,5114,10
3531005,3531012,43118,25183,7
4901408,4901415,60604,6377,9
428387,428387,6112,11887,7
5241989,5241996,65212,16706,8
...,...,...,...,...
2626458,2626458,31202,12883,6
4945058,4945065,61061,9617,7
5110427,5110434,63199,1726,9
14579,14579,614,889,7


In [53]:
pedrito = 65515

In [59]:
recomendaciones = predict(pedrito, model, df_animes,'ANIME_ID')
recomendaciones

,ANIME_ID,ANIME_NAME,EPISODES,URL_IMAGE,DURATION,SCORED,SCORED_BY,RANKS,POPULARITY,FAVORITES,SYNOPSIS,STATUS,START_DATE,FINISH_DATE,ID_CLASSIFICATION,ID_SOURCE,ID_TYPE
22,1,Cowboy Bebop,26,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.81,365465,27.0,38,39145,"Crime is timeless. By the year 2071, humanity ...",Finished Airing,1998-04-03,1999-04-24,1,0,1
263,18,Initial D Fourth Stage,24,https://myanimelist.cdn-dena.com/images/anime/...,27 min. per ep.,8.24,37675,276.0,1261,588,Takumi Fujiwara finally joins Ryousuke and Kei...,Finished Airing,2004-04-17,2006-02-18,0,1,1
1531,31,Neon Genesis Evangelion: Death &amp; Rebirth,1,https://myanimelist.cdn-dena.com/images/anime/...,1 hr. 41 min.,7.50,70425,1639.0,699,384,"In the year 2015, more than a decade has passe...",Finished Airing,1997-03-15,1997-03-15,1,0,0


In [73]:
usuario = df_animes[df_animes['ANIME_ID'].isin(df_rating[df_rating['user_id']==pedrito]['anime_id'])].head(10)
usuario

,ANIME_ID,ANIME_NAME,EPISODES,URL_IMAGE,DURATION,SCORED,SCORED_BY,RANKS,POPULARITY,FAVORITES,SYNOPSIS,STATUS,START_DATE,FINISH_DATE,ID_CLASSIFICATION,ID_SOURCE,ID_TYPE
1,5114,Fullmetal Alchemist: Brotherhood,64,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,9.25,733592,1.0,4,106895,After a horrific alchemy experiment goes wrong...,Finished Airing,2009-04-05,2010-07-04,1,1,1
3,9253,Steins;Gate,24,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,9.14,563857,5.0,8,92423,Eccentric scientist Rintarou Okabe has a never...,Finished Airing,2011-04-06,2011-09-14,0,2,1
18,12355,Ookami Kodomo no Ame to Yuki,1,https://myanimelist.cdn-dena.com/images/anime/...,1 hr. 57 min.,8.77,204744,34.0,148,7087,"Hana, a hard-working college student, falls in...",Finished Airing,2012-07-21,2012-07-21,2,0,0
38,19,Monster,74,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.69,118856,49.0,152,17399,"Dr. Kenzou Tenma, an elite neurosurgeon recent...",Finished Airing,2004-04-07,2005-09-28,4,1,1
67,22535,Kiseijuu: Sei no Kakuritsu,24,https://myanimelist.cdn-dena.com/images/anime/...,23 min. per ep.,8.52,410107,94.0,42,15963,"All of a sudden, they arrived: parasitic alien...",Finished Airing,2014-10-09,2015-03-26,1,1,1
69,12431,Uchuu Kyoudai,99,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.56,26666,80.0,881,2503,"On a fateful summer night in 2006, Mutta Nanba...",Finished Airing,2012-04-01,2014-03-22,0,1,1
92,12029,Uchuu Senkan Yamato 2199,26,https://myanimelist.cdn-dena.com/images/anime/...,25 min. per ep.,8.46,21150,131.0,1282,1228,Earth's once green hills and rich blue oceans ...,Finished Airing,2012-05-25,2013-10-25,0,0,2
95,14719,JoJo no Kimyou na Bouken (TV),26,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.41,187286,159.0,166,12556,The year is 1868; English nobleman George Joes...,Finished Airing,2012-10-06,2013-04-06,1,1,1
97,10165,Nichijou,26,https://myanimelist.cdn-dena.com/images/anime/...,23 min. per ep.,8.52,151872,96.0,136,12447,Nichijou primarily focuses on the daily antics...,Finished Airing,2011-04-03,2011-09-25,0,1,1
102,11981,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku n...,1,https://myanimelist.cdn-dena.com/images/anime/...,1 hr. 56 min.,8.47,112763,120.0,403,4922,The young girls of Mitakihara happily live the...,Finished Airing,2013-10-26,2013-10-26,0,0,0


-----------------

In [74]:
recomendaciones.to_csv(f'/content/recomendaciones_1.csv')
usuario.to_csv(f'/content/usuario_1.csv')

In [66]:
juanita = 65212

In [69]:
recomendaciones = predict(juanita, model, df_animes,'ANIME_ID').head(3)
recomendaciones

,ANIME_ID,ANIME_NAME,EPISODES,URL_IMAGE,DURATION,SCORED,SCORED_BY,RANKS,POPULARITY,FAVORITES,SYNOPSIS,STATUS,START_DATE,FINISH_DATE,ID_CLASSIFICATION,ID_SOURCE,ID_TYPE
150,57,Beck,26,https://myanimelist.cdn-dena.com/images/anime/...,25 min. per ep.,8.37,87024,181.0,414,5739,"Fourteen-year-old Yukio ""Koyuki"" Tanaka is a d...",Finished Airing,2004-10-07,2005-03-31,1,1,1
974,17,Hungry Heart: Wild Striker,52,https://myanimelist.cdn-dena.com/images/anime/...,23 min. per ep.,7.69,8885,1130.0,2967,204,As the younger brother of Japanese soccer star...,Finished Airing,2002-09-11,2003-09-10,0,1,1
1425,25,Sunabouzu,24,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,7.52,34377,1586.0,1008,638,"The Great Kanto Desert, a sweltering wasteland...",Finished Airing,2004-10-06,2005-03-30,1,1,1


In [70]:
df_animes[df_animes['ANIME_ID'].isin(df_rating[df_rating['user_id']==juanita]['anime_id'])].head(10)

,ANIME_ID,ANIME_NAME,EPISODES,URL_IMAGE,DURATION,SCORED,SCORED_BY,RANKS,POPULARITY,FAVORITES,SYNOPSIS,STATUS,START_DATE,FINISH_DATE,ID_CLASSIFICATION,ID_SOURCE,ID_TYPE
16,23273,Shigatsu wa Kimi no Uso,22,https://myanimelist.cdn-dena.com/images/anime/...,22 min. per ep.,8.88,413527,21.0,37,39664,"Kousei Arima is a child prodigy known as the ""...",Finished Airing,2014-10-10,2015-03-20,0,1,1
19,1575,Code Geass: Hangyaku no Lelouch,25,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.79,627740,30.0,9,63614,"In the year 2010, the Holy Empire of Britannia...",Finished Airing,2006-10-06,2007-07-29,1,0,1
43,20583,Haikyuu!!,25,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.65,278372,57.0,86,23743,"Ever since having witnessed the ""Little Giant""...",Finished Airing,2014-04-06,2014-09-21,0,1,1
224,16706,Kami nomi zo Shiru Sekai: Megami-hen,12,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.22,112936,291.0,399,1512,Having freed a myriad of women from the runawa...,Finished Airing,2013-07-09,2013-09-24,0,1,1
301,24833,Ansatsu Kyoushitsu (TV),22,https://myanimelist.cdn-dena.com/images/anime/...,23 min. per ep.,8.22,313058,289.0,59,9953,Tucked in the mountains near the elite Kunugig...,Finished Airing,2015-01-10,2015-06-20,0,1,1
449,22319,Tokyo Ghoul,12,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.00,659308,560.0,6,28883,A sinister threat is invading Tokyo: flesh-eat...,Finished Airing,2014-07-04,2014-09-19,1,1,1
773,22199,Akame ga Kill!,24,https://myanimelist.cdn-dena.com/images/anime/...,23 min. per ep.,7.80,475884,869.0,25,14475,Night Raid is the covert assassination branch ...,Finished Airing,2014-07-07,2014-12-15,1,1,1
857,16011,Tokyo Ravens,24,https://myanimelist.cdn-dena.com/images/anime/...,23 min. per ep.,7.73,112113,1052.0,286,1578,Onmyoudou magic was once a powerful technique ...,Finished Airing,2013-10-09,2014-03-26,0,5,1
888,10790,Kore wa Zombie Desu ka? of the Dead,10,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,7.71,140584,1081.0,293,687,Aikawa Ayumu was revived as a zombie by the cu...,Finished Airing,2012-04-05,2012-06-07,1,5,1
1216,11759,Accel World,24,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,7.53,236083,1545.0,103,3680,"Haruyuki Arita is an overweight, bullied middl...",Finished Airing,2012-04-07,2012-09-22,0,5,1


In [75]:
recomendaciones.to_csv(f'/content/recomendaciones_2.csv')
usuario.to_csv(f'/content/usuario_2.csv')

In [78]:
with open('/content/recomendacion.jdr', 'wb') as f: pickle.dump(model, f)

In [87]:
id_jojos = [ list(df_animes[df_animes['ANIME_NAME'] == v ]['ANIME_ID'])[0] for v in  df_animes['ANIME_NAME'].unique() if 'jojo' in v.lower()]

In [90]:
df_rating[df_rating['anime_id'].isin(id_jojos)]

,Unnamed: 0,user_id,anime_id,rating
5332163,5332170,66195,26055,8
5707176,5707183,70939,14719,9
3294112,3294112,40084,14719,8
3745301,3745308,45792,14719,7
1594050,1594050,20240,665,8
...,...,...,...,...
58692,58692,1635,26055,8
4633264,4633271,58233,14719,8
3151400,3151400,38384,666,7
5585585,5585592,69155,20899,9


In [92]:
jojos = 66195

In [93]:
recomendaciones = predict(jojos, model, df_animes,'ANIME_ID')
recomendaciones

,ANIME_ID,ANIME_NAME,EPISODES,URL_IMAGE,DURATION,SCORED,SCORED_BY,RANKS,POPULARITY,FAVORITES,SYNOPSIS,STATUS,START_DATE,FINISH_DATE,ID_CLASSIFICATION,ID_SOURCE,ID_TYPE
152,5,Cowboy Bebop: Tengoku no Tobira,1,https://myanimelist.cdn-dena.com/images/anime/...,1 hr. 54 min.,8.41,111344,157.0,438,706,"Another day, another bounty—such is the life o...",Finished Airing,2001-09-01,2001-09-01,1,0,0
211,30,Neon Genesis Evangelion,26,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.32,364012,223.0,48,41098,Fifteen years after a cataclysmic event known ...,Finished Airing,1995-10-04,1996-03-27,0,0,1
1531,31,Neon Genesis Evangelion: Death &amp; Rebirth,1,https://myanimelist.cdn-dena.com/images/anime/...,1 hr. 41 min.,7.50,70425,1639.0,699,384,"In the year 2015, more than a decade has passe...",Finished Airing,1997-03-15,1997-03-15,1,0,0


In [94]:
usuario = df_animes[df_animes['ANIME_ID'].isin(df_rating[df_rating['user_id']==jojos]['anime_id'])].head(10)
usuario

,ANIME_ID,ANIME_NAME,EPISODES,URL_IMAGE,DURATION,SCORED,SCORED_BY,RANKS,POPULARITY,FAVORITES,SYNOPSIS,STATUS,START_DATE,FINISH_DATE,ID_CLASSIFICATION,ID_SOURCE,ID_TYPE
35,431,Howl no Ugoku Shiro,1,https://myanimelist.cdn-dena.com/images/anime/...,1 hr. 57 min.,8.73,334114,43.0,89,9753,"That jumbled piece of architecture, that cacop...",Finished Airing,2004-11-20,2004-11-20,3,3,0
64,26055,JoJo no Kimyou na Bouken: Stardust Crusaders 2...,24,https://myanimelist.cdn-dena.com/images/anime/...,23 min. per ep.,8.56,121404,78.0,400,4468,Joutarou Kuujou and his allies have finally ma...,Finished Airing,2015-01-10,2015-06-20,1,1,1
86,16498,Shingeki no Kyojin,25,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.49,940211,110.0,2,64032,"Centuries ago, mankind was slaughtered to near...",Finished Airing,2013-04-07,2013-09-29,1,1,1
353,5682,Phantom: Requiem for the Phantom,26,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.11,83160,425.0,373,3688,Mafia is rife in America where assassinations ...,Finished Airing,2009-04-03,2009-09-25,1,2,1
411,1689,Byousoku 5 Centimeter,3,https://myanimelist.cdn-dena.com/images/anime/...,22 min. per ep.,7.95,280982,610.0,96,7832,What happens when two people love each other b...,Finished Airing,2007-03-03,2007-03-03,0,0,0
440,3588,Soul Eater,51,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,8.03,422552,523.0,23,17953,Death City is home to the famous Death Weapon ...,Finished Airing,2008-04-07,2009-03-30,0,1,1
507,3572,Macross F,25,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,7.98,51693,580.0,791,1743,Following a catastrophic war against a race of...,Finished Airing,2008-04-04,2008-09-26,0,0,1
649,10357,Jinrui wa Suitai Shimashita,12,https://myanimelist.cdn-dena.com/images/anime/...,23 min. per ep.,7.87,40680,756.0,723,1627,Because of the constantly declining birth rate...,Finished Airing,2012-07-02,2012-09-17,0,5,1
657,28999,Charlotte,13,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,7.92,315572,654.0,72,9561,If not for his ability to take over people's m...,Finished Airing,2015-07-05,2015-09-27,0,0,1
738,6213,Toaru Kagaku no Railgun,24,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,7.83,142516,835.0,220,3580,The student-filled Academy City is at the fore...,Finished Airing,2009-10-03,2010-03-20,0,1,1


In [95]:
recomendaciones.to_csv(f'/content/recomendaciones_2.csv')
usuario.to_csv(f'/content/usuario_2.csv')